# Chapter 20: Template Literal Types

---

## 20.1 Introduction to Template Literal Types

Template literal types extend the concept of string literal types to allow you to construct new string literal types by concatenating strings at the type level. They use the same syntax as JavaScript template literals but operate entirely within the type system, enabling powerful string manipulation and type-safe APIs.

### 20.1.1 Basic Template Literal Syntax

Template literal types use backticks (`` ` ``) with embedded type expressions using `${}` syntax.

```typescript
// Basic template literal type
type World = "world";
type Greeting = `hello ${World}`;  // "hello world"

// Combining multiple string literals
type Name = "John";
type Domain = "example.com";
type Email = `${Name}@${Domain}`;  // "John@example.com"

// With unions (creates all combinations)
type Color = "red" | "blue";
type Size = "small" | "large";
type Style = `${Color}-${Size}`;
// "red-small" | "red-large" | "blue-small" | "blue-large"
```

**How Template Literal Types Work:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                 Template Literal Type Expansion                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Type Definition:                                                  │
│   type Color = "red" | "blue";                                      │
│   type Size = "small" | "large";                                    │
│   type Variant = `${Color}-${Size}`;                                │
│                                                                     │
│   Expansion Process:                                                │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ 1. Take first union member of Color: "red"                   │ │
│   │    • "red-small" | "red-large"                               │ │
│   │                                                              │ │
│   │ 2. Take second union member of Color: "blue"                 │ │
│   │    • "blue-small" | "blue-large"                             │ │
│   │                                                              │ │
│   │ 3. Combine all results:                                      │ │
│   │    "red-small" | "red-large" | "blue-small" | "blue-large"   │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Result: A union of 4 specific string literal types                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 20.1.2 Constructing Complex String Types

Template literals can combine multiple types to create sophisticated string patterns.

```typescript
// Building URL patterns
type Protocol = "http" | "https";
type Subdomain = "www" | "api" | "admin";
type Domain = "example" | "company";
type TLD = "com" | "org" | "net";

type URL = `${Protocol}://${Subdomain}.${Domain}.${TLD}`;
// Creates 2 × 3 × 2 × 3 = 36 possible URL combinations!

// Example:
// "https://www.example.com"
// "http://api.company.org"
// "https://admin.example.net"

// CSS property generation
type Direction = "top" | "right" | "bottom" | "left";
type CSSProperty = `margin-${Direction}` | `padding-${Direction}`;
// "margin-top" | "margin-right" | "margin-bottom" | "margin-left"
// "padding-top" | "padding-right" | "padding-bottom" | "padding-left"

// Event naming conventions
type EventName = "click" | "focus" | "blur";
type HandlerName = `on${Capitalize<EventName>}`;
// "onClick" | "onFocus" | "onBlur"
```

**Practical Example - API Endpoints:**

```typescript
// Type-safe API endpoint definitions
type APIVersion = "v1" | "v2";
type Resource = "users" | "posts" | "comments";
type Action = "list" | "get" | "create" | "update" | "delete";

// Base endpoints
type Endpoint = `/api/${APIVersion}/${Resource}`;

// Specific action endpoints
type ActionEndpoint = 
  | `/api/${APIVersion}/${Resource}/${Action}`
  | `/api/${APIVersion}/${Resource}/${number}`;  // For specific IDs

// Usage
function fetchAPI(endpoint: Endpoint): Promise<any> {
  return fetch(endpoint);
}

// Valid calls:
fetchAPI("/api/v1/users");        // ✅ OK
fetchAPI("/api/v2/posts");        // ✅ OK

// Invalid calls:
// fetchAPI("/api/v3/users");     // ❌ Error: "v3" not assignable
// fetchAPI("/users");            // ❌ Error: missing prefix
```

### 20.1.3 Template Literals with Primitive Types

Template literal types can interpolate `string`, `number`, `bigint`, `boolean`, `null`, or `undefined`.

```typescript
// With string (accepts any string, not just literals)
type WithString = `ID-${string}`;  // Matches "ID-anything"

const valid1: WithString = "ID-123";
const valid2: WithString = "ID-abc";
const valid3: WithString = "ID-user-456";
// const invalid: WithString = "ID";  // ❌ Error: missing content after "ID-"

// With number (accepts any number)
type WithNumber = `Version-${number}`;

const v1: WithNumber = "Version-1";
const v2: WithNumber = "Version-3.14";
// const v3: WithNumber = "Version-latest";  // ❌ Error: must be number

// With boolean
type WithBoolean = `Enabled-${boolean}`;
// "Enabled-true" | "Enabled-false"

const b1: WithBoolean = "Enabled-true";
const b2: WithBoolean = "Enabled-false";

// With bigint
type WithBigint = `ID-${bigint}`;
const big: WithBigint = "ID-123456789012345678901234567890n";

// Combined
type Mixed = `User-${string}-Age-${number}`;
const user: Mixed = "User-John-Age-30";
```

---

## 20.2 String Manipulation Types

TypeScript provides built-in template literal types for common string transformations. These operate at the type level to transform string literal types.

### 20.2.1 `Uppercase<StringType>`

Converts each character in the string literal type to uppercase.

```typescript
type Uppercase<T extends string> = intrinsic;  // Built-in implementation

// Usage
type Greeting = "hello world";
type Shout = Uppercase<Greeting>;  // "HELLO WORLD"

// Practical use case - HTTP methods
type Method = "get" | "post" | "put" | "delete";
type UpperMethod = Uppercase<Method>;
// "GET" | "POST" | "PUT" | "DELETE"

// Environment variables (conventionally uppercase)
type EnvKey = "api_url" | "api_key";
type EnvVar = Uppercase<EnvKey>;
// "API_URL" | "API_KEY"

// Constants generation
type EventType = "user_created" | "user_updated";
type Constant = Uppercase<EventType>;
// "USER_CREATED" | "USER_UPDATED"
```

### 20.2.2 `Lowercase<StringType>`

Converts each character in the string literal type to lowercase.

```typescript
type Lowercase<T extends string> = intrinsic;  // Built-in implementation

// Usage
type HTTPMethod = "GET" | "POST";
type LowerMethod = Lowercase<HTTPMethod>;  // "get" | "post"

// Normalizing enum-like strings
type Input = "ACTIVE" | "INACTIVE" | "PENDING";
type Normalized = Lowercase<Input>;
// "active" | "inactive" | "pending"

// File extensions
type Ext = "PNG" | "JPG" | "GIF";
type LowerExt = Lowercase<Ext>;
// "png" | "jpg" | "gif"
```

### 20.2.3 `Capitalize<StringType>`

Converts the first character in the string literal type to uppercase.

```typescript
type Capitalize<T extends string> = intrinsic;  // Built-in implementation

// Usage
type Property = "name" | "age" | "email";
type GetterName = `get${Capitalize<Property>}`;
// "getName" | "getAge" | "getEmail"

type SetterName = `set${Capitalize<Property>}`;
// "setName" | "setAge" | "setEmail"

// React event handlers
type Event = "click" | "change" | "submit";
type Handler = `on${Capitalize<Event>}`;
// "onClick" | "onChange" | "onSubmit"

// Class method generation
type Field = "id" | "created_at";
type Method = `format${Capitalize<Field>}`;
// "formatId" | "formatCreated_at"
```

### 20.2.4 `Uncapitalize<StringType>`

Converts the first character in the string literal type to lowercase.

```typescript
type Uncapitalize<T extends string> = intrinsic;  // Built-in implementation

// Usage
type ClassName = "UserService" | "OrderController";
type Instance = Uncapitalize<ClassName>;
// "userService" | "orderController"

// Converting PascalCase to camelCase keys
type PascalKey = "UserName" | "UserEmail";
type CamelKey = Uncapitalize<PascalKey>;
// "userName" | "userEmail"

// Removing prefixes
type Prefixed = "getName" | "getAge";
type Unprefixed = Uncapitalize<Prefixed>;  // Doesn't remove prefix, just lowercases first letter
// Actually: "getName" | "getAge" (no change since 'g' is already lowercase)

// Better example:
type APIResponse = "Success" | "Error";
type CamelResponse = Uncapitalize<APIResponse>;
// "success" | "error"
```

**Complete String Transformation Pipeline:**

```typescript
// Combining all transformations
type Input = "user-profile-data";

// Step-by-step transformation
type Step1 = Capitalize<Input>;      // "User-profile-data"
type Step2 = Capitalize<Step1>;      // No change to rest of string
// Actually, Capitalize only affects first character:
// "User-profile-data"

// Real pipeline: snake_case to camelCase
type SnakeToCamel<S extends string> = 
  S extends `${infer Head}_${infer Tail}`
    ? `${Head}${Capitalize<SnakeToCamel<Tail>>}`
    : S;

type Test = SnakeToCamel<"user_profile_data">;  // "userProfileData"
type Test2 = SnakeToCamel<"api_key_secret">;    // "apiKeySecret"

// PascalCase conversion
type ToPascal<S extends string> = Capitalize<SnakeToCamel<S>>;
type Test3 = ToPascal<"user_profile">;  // "UserProfile"

// kebab-case to camelCase
type KebabToCamel<S extends string> =
  S extends `${infer Head}-${infer Tail}`
    ? `${Head}${Capitalize<KebabToCamel<Tail>>}`
    : S;

type Test4 = KebabToCamel<"user-profile-data">;  // "userProfileData"
```

---

## 20.3 Combining with Union Types

One of the most powerful features of template literal types is their interaction with union types. When a template literal contains a union type placeholder, TypeScript expands it to a union of all possible combinations.

### 20.3.1 Cartesian Product Generation

Template literals with multiple union slots create the Cartesian product of all combinations.

```typescript
// Two unions create combinations
type X = "a" | "b";
type Y = "1" | "2";
type Combo = `${X}-${Y}`;
// "a-1" | "a-2" | "b-1" | "b-2"

// Three unions create even more combinations
type Z = "x" | "y";
type Triple = `${X}-${Y}-${Z}`;
// 2 × 2 × 2 = 8 combinations:
// "a-1-x" | "a-1-y" | "a-2-x" | "a-2-y" | "b-1-x" | "b-1-y" | "b-2-x" | "b-2-y"

// Practical example - CSS grid areas
type Row = "header" | "main" | "footer";
type Col = "start" | "center" | "end";
type GridArea = `${Row}-${Col}`;
// "header-start" | "header-center" | "header-end"
// "main-start" | "main-center" | "main-end"
// "footer-start" | "footer-center" | "footer-end"

// BEM (Block Element Modifier) naming convention
type Block = "btn" | "card" | "modal";
type Element = "icon" | "text" | "container";
type Modifier = "primary" | "secondary" | "large" | "small";

type BEM = 
  | `${Block}`
  | `${Block}__${Element}`
  | `${Block}--${Modifier}`
  | `${Block}__${Element}--${Modifier}`;

// Generates:
// "btn", "card", "modal"
// "btn__icon", "btn__text", "card__container", etc.
// "btn--primary", "btn--secondary", etc.
// "btn__icon--primary", "btn__text--large", etc.
```

### 20.3.2 Selective Combination with Conditionals

Combine template literals with conditional types to filter combinations.

```typescript
// Filter valid combinations only
type Direction = "top" | "right" | "bottom" | "left";
type ValidPair = `${Direction}-${Exclude<Direction, "top" | "bottom">}`;
// "top-right" | "top-left" | "bottom-right" | "bottom-left"
// (Excludes "top-bottom", "bottom-top", etc.)

// Type-safe property paths with filtering
type ObjectPaths = "user" | "user.name" | "user.email" | "settings" | "settings.theme";

// Extract top-level only
type TopLevel<T extends string> = T extends `${infer _}.${infer _}` ? never : T;
type TopPaths = TopLevel<ObjectPaths>;  // "user" | "settings"

// Extract nested only
type Nested<T extends string> = T extends `${infer _}.${infer _}` ? T : never;
type NestedPaths = Nested<ObjectPaths>;  // "user.name" | "user.email" | "settings.theme"

// Conditional prefix addition
type AddPrefix<T extends string, P extends string> = 
  T extends `${P}${infer _}` ? T : `${P}${T}`;

type Prefixed = AddPrefix<"name" | "age" | "userName", "get">;
// "getName" | "getAge" | "userName" (already has prefix, unchanged)
```

### 20.3.3 Recursive Template Literals

Template literals can be recursive to handle nested structures.

```typescript
// Build paths recursively
type DeepPaths<T, Prefix extends string = ""> = 
  T extends object 
    ? { [K in keyof T]: K extends string 
        ? `${Prefix}${K}` | DeepPaths<T[K], `${Prefix}${K}.`>
        : never 
      }[keyof T]
    : never;

interface Config {
  server: {
    host: string;
    port: number;
    ssl: {
      enabled: boolean;
      cert: string;
    };
  };
  database: {
    url: string;
    pool: {
      min: number;
      max: number;
    };
  };
}

type Paths = DeepPaths<Config>;
// "server" | "server.host" | "server.port" | "server.ssl" | "server.ssl.enabled" | ...

// Flatten nested keys with delimiter
type FlattenKeys<T, Prefix extends string = ""> = 
  T extends object 
    ? { [K in keyof T]: K extends string 
        ? T[K] extends object 
          ? FlattenKeys<T[K], `${Prefix}${K}.`>
          : `${Prefix}${K}`
        : never 
      }[keyof T]
    : never;

type FlatPaths = FlattenKeys<Config>;
// "server.host" | "server.port" | "server.ssl.enabled" | "server.ssl.cert" | ...
```

---

## 20.4 Template Literal Type Inference

Just as conditional types can extract types with `infer`, template literals can parse and extract parts of string patterns.

### 20.4.1 Pattern Matching with `infer`

Extract substrings from template literal types using conditional types with `infer`.

```typescript
// Extract parts of a URL
type ParseURL<T extends string> = 
  T extends `${infer Protocol}://${infer Domain}/${infer Path}`
    ? { protocol: Protocol; domain: Domain; path: Path }
    : never;

type Parsed = ParseURL<"https://api.example.com/users">;
// { protocol: "https"; domain: "api.example.com"; path: "users" }

// Extract file extension
type GetExtension<T extends string> = 
  T extends `${infer Name}.${infer Ext}` ? Ext : never;

type Ext1 = GetExtension<"document.pdf">;   // "pdf"
type Ext2 = GetExtension<"archive.tar.gz">; // "gz" (last extension only)

// Get all extensions
type GetAllExtensions<T extends string> = 
  T extends `${infer Name}.${infer Rest}` 
    ? Rest extends `${infer _}.${infer _}` 
      ? GetAllExtensions<Rest> 
      : Rest
    : never;

type AllExt = GetAllExtensions<"archive.tar.gz">;  // "gz"

// Better: Get all segments
type SplitPath<T extends string> = 
  T extends `${infer Head}/${infer Tail}` 
    ? [Head, ...SplitPath<Tail>] 
    : [T];

type Segments = SplitPath<"api/v1/users/123">;
// ["api", "v1", "users", "123"]
```

### 20.4.2 Extracting and Transforming

Parse complex patterns and transform them into structured types.

```typescript
// Parse CSS property values
type ParseCSSValue<T extends string> = 
  T extends `${number}px` ? { type: "pixel"; value: number; unit: "px" }
  : T extends `${number}em` ? { type: "relative"; value: number; unit: "em" }
  : T extends `${number}%` ? { type: "percent"; value: number; unit: "%" }
  : { type: "unknown"; value: T };

type V1 = ParseCSSValue<"16px">;    // { type: "pixel"; value: number; unit: "px" }
type V2 = ParseCSSValue<"1.5em">;   // { type: "relative"; value: number; unit: "em" }
type V3 = ParseCSSValue<"100%">;    // { type: "percent"; value: number; unit: "%" }
type V4 = ParseCSSValue<"auto">;    // { type: "unknown"; value: "auto" }

// Extract query parameters
type ParseQuery<T extends string> = 
  T extends `${infer Key}=${infer Value}` 
    ? { key: Key; value: Value }
    : never;

type Query = ParseQuery<"page=1">;  // { key: "page"; value: "1" }

// Parse multiple parameters
type ParseQueryString<T extends string> = 
  T extends `${infer Param}&${infer Rest}`
    ? ParseQuery<Param> | ParseQueryString<Rest>
    : ParseQuery<T>;

type Queries = ParseQueryString<"page=1&limit=10&sort=asc">;
// { key: "page"; value: "1" } | { key: "limit"; value: "10" } | { key: "sort"; value: "asc" }
```

### 20.4.3 Advanced Parsing Patterns

Handle more complex parsing scenarios with recursive patterns.

```typescript
// Remove a prefix
type RemovePrefix<T extends string, P extends string> = 
  T extends `${P}${infer Rest}` ? Rest : T;

type NoPrefix = RemovePrefix<"getUser", "get">;  // "User"
type NoChange = RemovePrefix<"setUser", "get">;  // "setUser" (no match)

// Remove a suffix
type RemoveSuffix<T extends string, S extends string> = 
  T extends `${infer Rest}${S}` ? Rest : T;

type NoSuffix = RemoveSuffix<"userId", "Id">;  // "user"

// Replace substring
type Replace<T extends string, From extends string, To extends string> = 
  T extends `${infer Before}${From}${infer After}`
    ? `${Before}${To}${After}`
    : T;

type Replaced = Replace<"user-name", "-", "_">;  // "user_name"

// Replace all occurrences (recursive)
type ReplaceAll<T extends string, From extends string, To extends string> = 
  T extends `${infer Before}${From}${infer After}`
    ? `${Before}${To}${ReplaceAll<After, From, To>}`
    : T;

type AllReplaced = ReplaceAll<"user-name-here", "-", "_">;  // "user_name_here"

// Trim whitespace
type TrimLeft<T extends string> = 
  T extends ` ${infer Rest}` ? TrimLeft<Rest> : T;

type TrimRight<T extends string> = 
  T extends `${infer Rest} ` ? TrimRight<Rest> : T;

type Trim<T extends string> = TrimLeft<TrimRight<T>>;

type Trimmed = Trim<"  hello world  ">;  // "hello world"
```

---

## 20.5 Real-World Use Cases

Template literal types shine in real-world applications where string patterns need to be type-safe.

### 20.5.1 Type-Safe Event Names

Create type-safe event systems where event names follow patterns and carry typed payloads.

```typescript
// Event naming convention: domain:action
type Domain = "user" | "order" | "product";
type Action = "created" | "updated" | "deleted" | "viewed";

type EventName = `${Domain}:${Action}`;
// "user:created" | "user:updated" | ... | "product:viewed"

// Map event names to their payload types
type EventPayload = {
  "user:created": { userId: string; email: string };
  "user:updated": { userId: string; changes: Partial<{ email: string; name: string }> };
  "order:created": { orderId: string; total: number };
  // ... other mappings
};

// Type-safe event emitter
class TypedEmitter<Events extends Record<string, any>> {
  emit<K extends keyof Events>(event: K, payload: Events[K]): void {
    console.log(`Emitting ${String(event)}`, payload);
  }

  on<K extends keyof Events>(
    event: K, 
    handler: (payload: Events[K]) => void
  ): void {
    // Implementation...
  }
}

const emitter = new TypedEmitter<EventPayload>();

emitter.emit("user:created", { userId: "123", email: "test@example.com" }); // ✅ OK
// emitter.emit("user:created", { userId: "123" }); // ❌ Error: missing email
// emitter.emit("user:unknown", {}); // ❌ Error: event doesn't exist
```

### 20.5.2 Type-Safe Routing

Implement type-safe routing systems for frontend or backend applications.

```typescript
// Route parameters extraction
type RouteParams<T extends string> = 
  T extends `${infer _}:${infer Param}/${infer Rest}`
    ? { [K in Param | keyof RouteParams<Rest>]: string }
    : T extends `${infer _}:${infer Param}`
      ? { [K in Param]: string }
      : {};

type UserRoute = "/users/:userId/posts/:postId";
type Params = RouteParams<UserRoute>;
// { userId: string; postId: string }

// Type-safe router
class Router<T extends string> {
  private routes: Map<T, Function> = new Map();

  addRoute<P extends T>(
    path: P, 
    handler: (params: RouteParams<P>) => void
  ): void {
    this.routes.set(path, handler);
  }

  navigate<P extends T>(path: P, params: RouteParams<P>): void {
    const handler = this.routes.get(path);
    if (handler) handler(params);
  }
}

const router = new Router<"/users/:id" | "/posts/:postId">();

router.addRoute("/users/:id", (params) => {
  console.log(params.id);  // TypeScript knows params has 'id'
});

router.navigate("/users/:id", { id: "123" }); // ✅ OK
// router.navigate("/users/:id", { userId: "123" }); // ❌ Error: wrong param name
```

### 20.5.3 CSS-in-JS Type Safety

Provide type safety for CSS-in-JS libraries with autocompletion for CSS properties.

```typescript
// Theme-aware CSS properties
type Spacing = "sm" | "md" | "lg" | "xl";
type Color = "primary" | "secondary" | "danger" | "success";

type ThemeKey = 
  | `spacing-${Spacing}`
  | `color-${Color}`
  | `color-${Color}-light`
  | `color-${Color}-dark`;

// CSS Property with theme values
type CSSWithTheme = {
  margin?: Spacing | `var(--spacing-${Spacing})`;
  padding?: Spacing | `var(--spacing-${Spacing})`;
  color?: Color | `var(--color-${Color})`;
  backgroundColor?: Color | `var(--color-${Color})`;
};

const styles: CSSWithTheme = {
  margin: "sm",                    // ✅ OK
  padding: "var(--spacing-lg)",    // ✅ OK
  color: "primary",                // ✅ OK
  // backgroundColor: "unknown",   // ❌ Error
};

// Strict CSS class names
type BEMBlock = "btn" | "card" | "input";
type BEMElement = "icon" | "label" | "container";
type BEMModifier = "primary" | "secondary" | "large" | "small" | "disabled";

type ClassName = 
  | `${BEMBlock}`
  | `${BEMBlock}__${BEMElement}`
  | `${BEMBlock}--${BEMModifier}`
  | `${BEMBlock}__${BEMElement}--${BEMModifier}`;

function classNames(...classes: ClassName[]): string {
  return classes.join(" ");
}

// Valid usage
const classes = classNames(
  "btn",
  "btn__icon",
  "btn--primary",
  "btn__icon--large"
);
```

### 20.5.4 Environment Variables and Configuration

Type-safe access to environment variables with naming conventions.

```typescript
// Type-safe env var access
type Prefix = "VITE_" | "REACT_APP_" | "NEXT_PUBLIC_";
type EnvVar = 
  | `${Prefix}API_URL`
  | `${Prefix}API_KEY`
  | `${Prefix}DATABASE_URL`
  | `${Prefix}STRIPE_KEY`;

// Runtime validation with types
function getEnvVar<T extends EnvVar>(key: T): string {
  const value = process.env[key];
  if (!value) throw new Error(`Missing environment variable: ${key}`);
  return value;
}

// Usage
const apiUrl = getEnvVar("VITE_API_URL");     // ✅ OK
const apiKey = getEnvVar("REACT_APP_API_KEY"); // ✅ OK
// const invalid = getEnvVar("UNKNOWN_VAR");  // ❌ Error

// Configuration keys with nesting
type ConfigSection = "db" | "cache" | "api";
type ConfigKey = "host" | "port" | "timeout" | "retries";

type ConfigPath = `${ConfigSection}.${ConfigKey}`;
// "db.host" | "db.port" | "db.timeout" | ... | "api.retries"

type Config = {
  [K in ConfigPath]: K extends `db.${infer _}` ? string | number 
    : K extends `cache.${infer _}` ? number 
    : string;
};

const config: Config = {
  "db.host": "localhost",
  "db.port": 5432,
  "cache.timeout": 3000,
  "api.host": "https://api.example.com"
};
```

### 20.5.5 Internationalization (i18n)

Type-safe translation keys with parameter interpolation.

```typescript
// Translation keys with parameters
type TranslationKey = 
  | "welcome"
  | "hello_user"
  | "items_count"
  | "user_profile_title";

// Define which keys need parameters
type TranslationParams = {
  "welcome": {};
  "hello_user": { username: string };
  "items_count": { count: number };
  "user_profile_title": { firstName: string; lastName: string };
};

// Type-safe translation function
function t<K extends TranslationKey>(
  key: K,
  ...args: TranslationParams[K] extends Record<string, never> 
    ? [] 
    : [params: TranslationParams[K]]
): string {
  // Implementation would fetch the translation and interpolate
  return key;
}

// Usage
t("welcome");                                    // ✅ OK
t("hello_user", { username: "John" });           // ✅ OK
t("items_count", { count: 5 });                  // ✅ OK
// t("hello_user");                              // ❌ Error: missing params
// t("hello_user", { count: 5 });                // ❌ Error: wrong params
```

---

## 20.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored template literal types, TypeScript's tool for string manipulation at the type level:

**Key Takeaways:**

1. **Basic Template Literals**:
   - Syntax: `` `hello ${World}` ``
   - Combines string literals to create new literal types
   - Distributes over union types to create all combinations

2. **String Manipulation Types**:
   - `Uppercase<T>` - Convert to uppercase
   - `Lowercase<T>` - Convert to lowercase
   - `Capitalize<T>` - Capitalize first letter
   - `Uncapitalize<T>` - Lowercase first letter
   - Can be composed for complex transformations (snake_case to camelCase)

3. **Union Type Expansion**:
   - Cartesian product generation with multiple unions
   - `${A}-${B}` creates all combinations of A and B
   - Useful for generating CSS classes, API endpoints, event names

4. **Type Inference**:
   - Use `infer` with template literals to parse strings
   - Extract substrings, prefixes, suffixes
   - Parse structured patterns (URLs, file paths, query strings)

5. **Real-World Applications**:
   - Type-safe event systems with pattern-based names
   - Route parameter extraction for routers
   - CSS-in-JS type safety
   - Environment variable validation
   - Internationalization with typed parameters

### Practical Exercises

**Exercise 1: Basic Template Literals**

Create the following type utilities:

```typescript
// 1. Create a type that generates all HTTP methods with /api prefix
//    Result: "/api/get" | "/api/post" | "/api/put" | "/api/delete"

// 2. Create a type for CSS margin/padding with all directions
//    Result: "margin-left" | "margin-right" | ... | "padding-bottom"

// 3. Create a type for status codes with category prefix
//    Input: "success" | "error", Code: 200 | 404 | 500
//    Result: "success-200" | "success-404" | ... | "error-500"

// 4. Create file path types that combine directory and filename
//    Dir: "src" | "dist", File: "index.ts" | "utils.ts"
//    Result: "src/index.ts" | "src/utils.ts" | "dist/index.ts" | "dist/utils.ts"
```

**Exercise 2: String Transformations**

Implement string transformation utilities:

```typescript
// 1. Convert snake_case to camelCase
type SnakeToCamel<"user_profile_data"> = "userProfileData"

// 2. Convert camelCase to snake_case (reverse operation)
type CamelToSnake<"userProfileData"> = "user_profile_data"

// 3. Create a type that adds "get" prefix and capitalizes the property
type CreateGetter<"name" | "age"> = "getName" | "getAge"

// 4. Remove a specific prefix from a string type
type RemovePrefix<"IUser" | "IOrder", "I"> = "User" | "Order"
```

**Exercise 3: Type Parsing**

Parse complex string patterns:

```typescript
// 1. Extract the protocol from a URL
type GetProtocol<"https://example.com"> = "https"

// 2. Extract filename and extension separately
type ParseFile<"document.pdf"> = { name: "document"; ext: "pdf" }

// 3. Parse a query string into an object type
type ParseQuery<"page=1&limit=10"> = { page: "1"; limit: "10" }

// 4. Extract route parameters from a path
type GetParams<"/users/:userId/posts/:postId"> = "userId" | "postId"
```

**Exercise 4: Advanced Patterns**

Build complex utility types:

```typescript
// 1. Create a type-safe event system where:
//    - Events follow pattern: "domain:action" (e.g., "user:create")
//    - Each event has a specific payload type
//    - The emit function is fully typed

// 2. Build a type-safe CSS class generator using BEM methodology:
//    - Block: "btn", "card"
//    - Element: "__icon", "__text"
//    - Modifier: "--primary", "--large"
//    - Valid combinations: "btn", "btn__icon", "btn--primary", "btn__icon--primary"

// 3. Create a path extraction type that:
//    - Takes an object type
//    - Generates all possible dot-notation paths (e.g., "user.name", "user.address.city")
//    - Works recursively for nested objects

// 4. Implement a sprintf-style type:
//    - Input: "Hello %s, you have %d messages"
//    - Extracts: [string, number] as the required argument types
```

**Exercise 5: Real-World Application**

Build a type-safe router configuration:

```typescript
// Define routes with parameters
type RouteConfig = {
  "/": {};
  "/users/:userId": { userId: string };
  "/users/:userId/posts/:postId": { userId: string; postId: string };
  "/search": { query: string; page?: number };
};

// Implement:
// 1. A type that extracts all route paths
// 2. A type that extracts parameters for a given route
// 3. A navigate function that:
//    - Accepts only valid routes
//    - Requires correct parameters for the route
//    - Validates parameter types

// Example usage:
navigate("/"); // ✅ OK
navigate("/users/:userId", { userId: "123" }); // ✅ OK
navigate("/users/:userId", { userId: 123 }); // ❌ Error: userId should be string
navigate("/unknown"); // ❌ Error: invalid route
```

### Additional Resources

- **TypeScript Handbook - Template Literal Types**: https://www.typescriptlang.org/docs/handbook/2/template-literal-types.html
- **Total TypeScript - Template Literals**: https://www.totaltypescript.com/tips/template-literal-types
- **Type Challenges**: https://github.com/type-challenges/type-challenges (Medium/Hard string manipulation challenges)
- **Advanced TypeScript**: https://github.com/microsoft/TypeScript-New-Handbook/blob/master/reference/Template%20Literal%20Types.md

---

## Coming Up Next: Chapter 21 - Utility Types

In the next chapter, we will explore **Utility Types**, TypeScript's built-in type transformations and how to create your own:

- **21.1 Overview of Utility Types** - Understanding TypeScript's built-in toolbox
- **21.2 Property Modifiers** - `Partial`, `Required`, `Readonly` deep dive
- **21.3 Type Transformations** - `Record`, `Pick`, `Omit` usage patterns
- **21.4 Union Manipulation** - `Exclude`, `Extract`, `NonNullable` in practice
- **21.5 Function Types** - `Parameters`, `ReturnType`, `InstanceType` applications
- **21.6 Other Utility Types** - `ThisParameterType`, `OmitThisParameter`, `ThisType`
- **21.7 Creating Custom Utility Types** - Building your own type transformation library

Utility types combine mapped types, conditional types, and template literals to provide a rich set of type transformations. Understanding these utilities—and how they're implemented—will complete your mastery of TypeScript's type system.